<a href="https://colab.research.google.com/github/SupradeepDanturti/ConvAIProject/blob/dev2/interface_test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install speechbrain

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
## drive link -> https://drive.google.com/file/d/1xfG7KJlP0SgriIE1BLpTrBhytACKIkAY/view?usp=sharing
!unzip /content/drive/MyDrive/ConvAI/interface_files.zip

Mounted at /content/drive/
Archive:  /content/drive/MyDrive/ConvAI/interface_files.zip
  inflating: classifier.ckpt         
  inflating: embedding_model.ckpt    
  inflating: hyperparams.yaml        
  inflating: label_encoder.txt       
  inflating: session_2_spk_2_mixture.wav  
  inflating: session_4_spk_1_mixture_004_segment.wav  


In [3]:
import torch
from speechbrain.inference.interfaces import Pretrained
import torchaudio
import math
from speechbrain.utils.data_utils import split_path
from speechbrain.utils.fetching import fetch

class SpeakerCounter(Pretrained):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.sample_rate = self.hparams.sample_rate

    MODULES_NEEDED = [
        "compute_features",
        "mean_var_norm",
        "embedding_model",
        "classifier",
    ]

    def resample_waveform(self, waveform, orig_sample_rate):
        """
        Resample the waveform to a new sample rate.
        """
        if orig_sample_rate != self.sample_rate:
            resample_transform = torchaudio.transforms.Resample(orig_freq=orig_sample_rate, new_freq=self.sample_rate)
            waveform = resample_transform(waveform)
        return waveform

    def merge_overlapping_segments(self, segments):
      if not segments:
          return []

      # Start with the first segment
      merged = [segments[0]]
      for current in segments[1:]:
          prev = merged[-1]
          if current[0] <= prev[1]:  # Check if the current segment overlaps with the previous
              # Merge the two segments if they have the same label
              if current[2] == prev[2]:
                  merged[-1] = (prev[0], max(prev[1], current[1]), prev[2])
              else:
                  merged.append(current)
          else:
              merged.append(current)
      return merged

    def aggregate_segments_with_overlap(self, segment_predictions):
        aggregated_predictions = []
        last_start, last_end, last_label = segment_predictions[0]

        for start, end, label in segment_predictions[1:]:
            if label == last_label and start <= last_end:
                # Extend the current segment if it overlaps and has the same label
                last_end = max(last_end, end)
            else:
                # Save the current segment and start a new one
                aggregated_predictions.append((last_start, last_end, last_label))
                last_start, last_end, last_label = start, end, label

        # Add the last segment
        aggregated_predictions.append((last_start, last_end, last_label))

        # Merge any further overlapped segments
        merged = self.merge_overlapping_segments(aggregated_predictions)
        return merged


    def analyze_transition(self, segment, start, end):

        # Define the size of sub-segments for detailed analysis, e.g., 0.25 seconds
        sub_segment_length = 0.25
        sample_rate = self.sample_rate

        sub_segment_samples = int(sub_segment_length * sample_rate)

        label_counts = {}
        max_label = None
        max_count = 0

        for sub_start in range(0, segment.shape[-1] - sub_segment_samples, sub_segment_samples):
            sub_end = sub_start + sub_segment_samples
            sub_segment = segment[:, sub_start:sub_end]

            emb = self.encode_batch(sub_segment)
            out_prob = self.mods.classifier(emb).squeeze(1)
            score, index = torch.max(out_prob, dim=-1)
            label = self.hparams.label_encoder.decode_torch(index)[0]

            # Update label counts
            label_counts[label] = label_counts.get(label, 0) + 1
            if label_counts[label] > max_count:
                max_label = label
                max_count = label_counts[label]
        return start, end, max_label


    def refine_transitions(self, segments, segment_predictions):
        """Refined to analyze transitions more precisely"""
        refined_predictions = []
        for i in range(len(segment_predictions) - 1):
            start_time, end_time, prediction = segment_predictions[i]
            next_start_time, next_end_time, next_prediction = segment_predictions[i + 1]

            if prediction != next_prediction:
                transition_start = max(start_time, next_start_time - 0.25)
                transition_end = min(end_time, next_start_time + 0.25)
                refined_prediction = self.analyze_transition(segments[i], transition_start, transition_end)
                refined_predictions.append(refined_prediction)
            else:
                refined_predictions.append(segment_predictions[i])

        if segment_predictions and (len(refined_predictions) < len(segment_predictions)):
            refined_predictions.append(segment_predictions[-1])

        return refined_predictions

    def encode_batch(self, wavs, wav_lens=None, normalize=False):
        # Manage single waveforms in input
        if len(wavs.shape) == 1:
            wavs = wavs.unsqueeze(0)

        if wav_lens is None:
            wav_lens = torch.ones(wavs.shape[0], device=self.device)

        wavs, wav_lens = wavs.to(self.device), wav_lens.to(self.device)
        wavs = wavs.float()

        # Computing features and embeddings
        feats = self.mods.compute_features(wavs)
        feats = self.mods.mean_var_norm(feats, wav_lens)
        embeddings = self.mods.embedding_model(feats, wav_lens)
        return embeddings

    def create_segments(self, waveform, segment_length, overlap):
        num_samples = waveform.shape[1]
        segment_samples = int(segment_length * self.sample_rate)
        overlap_samples = int(overlap * self.sample_rate)
        step_samples = segment_samples - overlap_samples
        segments = []
        segment_times = []

        for start in range(0, num_samples - segment_samples + 1, step_samples):
            end = start + segment_samples
            segments.append(waveform[:, start:end])
            start_time = start / self.sample_rate
            end_time = end / self.sample_rate
            segment_times.append((start_time, end_time))

        return segments, segment_times

    def classify_file(self, path, segment_length=2.0, overlap=1.0, **kwargs):
        """Adjusted to handle overlapped segment predictions and refining transitions"""
        waveform, osr = torchaudio.load(path)
        waveform = self.resample_waveform(waveform, osr)

        segments, segment_times = self.create_segments(waveform, segment_length, overlap)
        segment_predictions = []

        for segment, (start_time, end_time) in zip(segments, segment_times):
            rel_length = torch.tensor([1.0])
            emb = self.encode_batch(segment, rel_length)
            out_prob = self.mods.classifier(emb).squeeze(1)
            score, index = torch.max(out_prob, dim=-1)
            text_lab = self.hparams.label_encoder.decode_torch(index)
            segment_predictions.append((start_time, end_time, text_lab[0]))

        # refined_segment_predictions = self.refine_transitions(segments, segment_predictions)
        aggregated_predictions = self.aggregate_segments_with_overlap(segment_predictions)

        with open("sample_segment_predictions.txt", "w") as file:
            for start_time, end_time, prediction in aggregated_predictions:
                speaker_text = "no speech" if str(prediction) == "0" else ("1 speaker" if str(prediction) == "1" else f"{prediction} speakers")
                print(f"{start_time:.2f}-{end_time:.2f} has {speaker_text}")
                file.write(f"{start_time:.2f}-{end_time:.2f} has {speaker_text}\n")

    def forward(self, wavs, wav_lens=None):
        """Runs the classification"""
        return self.classify_file(wavs, wav_lens)

In [4]:
# from SpeakerCounter import SpeakerCounter
wav_path = "/content/session_2_spk_2_mixture.wav"
save_dir = "/content/SaveECAPAsampleinterface"
model_path = "/content"

# Instantiate your class using from_hparams
audio_classifier = SpeakerCounter.from_hparams(source=model_path, savedir=save_dir)

audio_classifier.classify_file(wav_path)

0.00-2.00 has 1 speaker
1.00-14.00 has 2 speakers
13.00-15.00 has 1 speaker
14.00-17.00 has 2 speakers
16.00-19.00 has 1 speaker
18.00-60.00 has 2 speakers
59.00-61.00 has 1 speaker
60.00-65.00 has 2 speakers
64.00-66.00 has 1 speaker
65.00-97.00 has 2 speakers
96.00-99.00 has 1 speaker
98.00-102.00 has 2 speakers
101.00-105.00 has 1 speaker
104.00-106.00 has 2 speakers
105.00-108.00 has 1 speaker
107.00-120.00 has 2 speakers
119.00-145.00 has 1 speaker


In [ ]:
!rm -rf /content/SaveECAPAsampleinterface

In [ ]:
%%file SpeakerCounter.py

import torch
from speechbrain.inference.interfaces import Pretrained
import torchaudio
import math
from speechbrain.utils.data_utils import split_path
from speechbrain.utils.fetching import fetch

class SpeakerCounter(Pretrained):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.sample_rate = self.hparams.sample_rate

    MODULES_NEEDED = [
        "compute_features",
        "mean_var_norm",
        "embedding_model",
        "classifier",
    ]
    def encode_batch(self, wavs, wav_lens=None, normalize=False):


        # Computing features and embeddings
        feats = self.mods.compute_features(wavs)
        feats = self.mods.mean_var_norm(feats, wav_lens)
        embeddings = self.mods.embedding_model(feats, wav_lens)
        return embeddings

    def classify_batch(self, wavs, wav_lens=None):
        emb = self.encode_batch(wavs, wav_lens)
        out_prob = self.mods.classifier(emb).squeeze(1)
        score, index = torch.max(out_prob, dim=-1)
        # text_lab = self.hparams.label_encoder.decode_torch(index)
        return out_prob, score, index
        # return out_prob, score, index, text_lab

    def classify_file(self, path, **kwargs):
        waveform = self.load_audio(path, **kwargs)
        # Fake a batch:
        batch = waveform.unsqueeze(0)
        rel_length = torch.tensor([1.0])
        emb = self.encode_batch(batch, rel_length)
        out_prob = self.mods.classifier(emb).squeeze(1)
        score, index = torch.max(out_prob, dim=-1)
        text_lab = self.hparams.label_encoder.decode_torch(index)
        return out_prob, score, index, text_lab

    def forward(self, wavs, wav_lens=None):
        """Runs the classification"""
        return self.classify_batch(wavs, wav_lens)

Overwriting SpeakerCounter.py


In [ ]:
from SpeakerCounter import SpeakerCounter
wav_path = "/content/session_2_spk_2_mixture.wav"
# wav_path = "/content/session_4_spk_1_mixture_004_segment.wav"
save_dir = "/content/SaveECAPAsampleinterface"
model_path = "/content"

# Instantiate your class using from_hparams
audio_classifier = SpeakerCounter.from_hparams(source=model_path, savedir=save_dir)

# audio_classifier.hparams.label_encoder.ignore_len()
# signal, fs = torchaudio.load(wav_path)
# # pred = audio_classifier.classify_file(wav_path)
# embeddings = audio_classifier.encode_batch(signal)
# prediction = audio_classifier.classify_batch(signal)
# print(prediction)

"""or """
audio_classifier.classify_file(wav_path)

RuntimeError: Failed to open the input "session_2_spk_2_mixture.wav" (Too many levels of symbolic links).
Exception raised from get_input_format_context at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/stream_reader/stream_reader.cpp:42 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7da782cced87 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x7da782c7f75f in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42904 (0x7da7828ca904 in /usr/local/lib/python3.10/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: torio::io::StreamingMediaDecoder::StreamingMediaDecoder(std::string const&, std::optional<std::string> const&, std::optional<std::map<std::string, std::string, std::less<std::string>, std::allocator<std::pair<std::string const, std::string> > > > const&) + 0x

In [ ]:
import torchaudio
def divide_audio_into_segments(audio_path, segment_length=2):
    # Load the audio file
    sample_rate = 16000
    waveform, _ = torchaudio.load(audio_path)

    # Calculate the number of samples for the given segment length
    num_samples_per_segment = sample_rate * segment_length

    # Calculate the total number of segments, using standard Python operations for ceiling
    total_segments = int(-(-waveform.size(1) // num_samples_per_segment))  # Ceiling division

    # Process and save each segment
    for i in range(total_segments):
        # Calculate the start and end sample for the current segment
        start_sample = i * num_samples_per_segment
        end_sample = start_sample + num_samples_per_segment

        # If the end sample exceeds the waveform length, adjust it to the waveform length
        end_sample = min(end_sample, waveform.size(1))

        # Extract the segment
        segment = waveform[:, start_sample:end_sample]

        # Save the segment to a file
        segment_file_name = f'/content/samples/segment_{i + 1}.wav'
        torchaudio.save(segment_file_name, segment, sample_rate)

divide_audio_into_segments(wav_path)